In [1]:
import numpy as np

## Ring Flange Stress


We have Flat Face Flanges with Metal-to-Metal Contact Outside the Bolt Circle, and therefore should use non-Mandatory Appendix Y, Class 3 Flange Assembly, Figure Y-5.1.3.  (For a large central hole in a flat plate where the hole is greater than 1/2 the flange diameter, the rules of Mandatory Appendix 14 should be used.)

SLArpaas material: stainless steel 304

### Flange parameters


$A$ = outside diameter of flange or, where slotted holes extend to the outside of the flange, the diameter to the bottom of the slots = 18 in

$B$ = inside diameter of flange = 14 in

$C$ = bolt‐circle diameter = 17 in

$G$ = 15.086 in (option b, when $b_o$ > 1/4 in. (6 mm), G = outside diameter of gasket contact face less 2b), from the Cryofab analysis

$t$ = flange thickness = 0.63 in (based on the Cryofab analysis and drawing)  Note that it is the thickness of the flange of the dewar, not of the top plate.

$n$ = number of bolts = 8

$P$ = 15 psig (internal design pressure)

$B_s$ = real bolt spacing = $C/n$ or the real chord length of the bolt spacing: $B_s = \frac{C}{2}\frac{\sin(2\pi/n)}{\sin((\pi-(2\pi/n))/2)}$

In [2]:
A = 18
B = 14
C = 17
G = 15.086
t = 0.63
n = 8
P = 15

### Bolt load

Table 2.5-1: 

> Self‐energizing types (O-rings, metallic, elastomer, other gasket 0 types considered as self‐sealing)

Mandatory Appendix 2-5 (c)(3)

> Bolt loads for flanges using gaskets of the self‐ energizing type differ from those shown above.

2-5 (c)(3)(-a) 

$W_{m1}$ = minimum required bolt load for the operating conditions

$W_{m1} = H+H_p$ (eq. 2.5 (c)(1)(1)), where $H_p = 0$ 

$H$ = total hydrostatic end force

$H = 0.785 G^2P$ 

2-5 (c)(3)(-b)

$W_{m2}$ = minimum required bolt load for gasket seating

$W_{m2}$ = 0

In [3]:
H = 0.785*(G**2)*P
Wm1 = H
print( f'ASME: Minimal required bolt load for the operating conditions: {Wm1} pound')

ASME: Minimal required bolt load for the operating conditions: 2679.8415879000004 pound


SLArpaas vessel < 60 pounds

Gross capacity of the SLArpaas vessel = 60 L

LAr density = 1.4 kg/L

LAr mass when SLArpaas is full = 1.4 kg/L * 60L

In [4]:
M_SLArpaas_lbs = 60
D_LAr_SI = 1.4
Vol_SLArpaas_L = 60
M_LAr_SI = D_LAr_SI*Vol_SLArpaas_L
M_LAr_lbs = 2.2*M_LAr_SI
Wm1 += M_SLArpaas_lbs + M_LAr_lbs
print( f'Minimal required bolt load for the operating conditions with the vessel and LAr weight: {Wm1} pound')

Minimal required bolt load for the operating conditions with the vessel and LAr weight: 2924.6415879000006 pound


### Bolt Spacing

2-5 (d)

$B_{s,max}$ = Maximum bolt spacing

2-5 (d)(3)

$B_{s,max} = 2a + \frac{6t}{m+0.5}$

$a$ = nominal bolt diameter, 0.313 in for 5/16"-18 bolts

$m$ = gasket factor, obtain from Table 2-5.1 [see Note in 2-5(c)(1)]

Table 2-5.1: $m$ = 0 for O-ring

Calculate the chord length of the bolt spacing: $B_s = \frac{C}{2}\frac{\sin(2\pi/n)}{\sin((\pi-(2\pi/n))/2)}$

In [5]:
a = 0.313
Bsmax = 2*a+ 6*t/0.5
Bs = C*np.sin(np.pi/4)/(2*np.sin(3*np.pi/8))
BsSatisfied = False
if Bsmax > Bs:
    BsSatisfied = True
print( f'Maximum bolt spacing should be {Bsmax} in, bolt spacing is {Bs} in, satisfied: {BsSatisfied}')

Maximum bolt spacing should be 8.186 in, bolt spacing is 6.505618350206526 in, satisfied: True


### Bolt Area

$S_T$ = specified minimum tensile strength at room temperature.  The bolts on McMaster are mostly made of SAE J429.  The Grade 2 (low or medium carbon steel) with 1/4" - 3/4" size has the minimum tensile strength 74000 psi.  The Cryofab analysis says 70000 psi and I think this is what they meant.

$S_b$ = allowable bolt stress at design temperature (see UG-23).  The Cryofab analysis show $S_b = S_T/4$ and I found the allowable stress value = $S_T/4$ in Table 6-100(a), 6-100(b), 6-100(c) in Section II Part D

$A_{m1}$ = total cross‐sectional area of bolts at root of thread or section of least diameter under stress,
required for the operating conditions
= $W_{m1}/S_b$

In [6]:
S_T = 70000
S_b = S_T/4.
Am1 = Wm1/S_b
print( f'Minimum required bolt area: {Am1} sq in')

Minimum required bolt area: 0.1671223764514286 sq in


$A_b$ = area of a bolt = 0.045 sq in

Ab_slarpaas = total bolt area in SLArpaas

In [7]:
Ab = 0.045
Ab_slarpaas = n*Ab

BoltAreaOK = False
if Ab_slarpaas > Am1:
    BoltAreaOK = True

print( f'Total bolt area in SLArpaas: {Ab_slarpaas} sq in; bolt area OK? {BoltAreaOK}')

Total bolt area in SLArpaas: 0.36 sq in; bolt area OK? True


### Flange Design Bolt Load

Mandatory Appendix 2-5(e)

$W$ = Flange design bolt load

$W = W_{m1}$ for operation conditions

$W = \frac{(A_m+A_b)S_a}{2}$ for gasket seating

where $S_a$ = allowable bolt stress at atmospheric temperature (see UG-23).  
Also see the explanation in 2-5(e).

In our case, $S_a = S_b$, $A_m = A_{m1}$ 

In [8]:
S_a = S_b
Am = Am1
W = (Am+Ab_slarpaas)*S_a/2
print( f'Flange design bolt load: {W} pound')

Flange design bolt load: 4612.32079395 pound


### Flange Moment


For the operating conditions, the total flange moment is described in Appendix 2-6,

$M_{op} = M_D+M_T+M_G$

where $M_{op}$ has the same notation in the Cryofab analysis ($M_o$ in ASME), $M_D = H_Dh_D$, $M_T = H_Th_T$ and $M_G = H_Gh_G$.

$H_D$ = hydrostatic end force on area inside of flange = $0.785B^2P$

$h_D$ = radial distance from the bolt circle, to the circle on which $H_D$ acts, as prescribed in Table 2-6 
= $(C-B)/2$

$H_T$ = difference between total hydrostatic end force and the hydrostatic end force on area inside of flange
= $H − H_D$

$h_T$ = radial distance from the bolt circle to the circle on which $H_T$ acts as prescribed in Table 2-6 
= $C/2 - (B/2+G/2)/2$

$H_G$ = gasket load for the operating condition
= $W_{m1} − H$

$h_G$ = radial distance from gasket load reaction to the bolt circle
= $(C−G)/2$

In [9]:
HD = 0.785*(B**2)*P
hD = (C-B)/2
HT = H - HD
hT = C/2 - (B/2 + G/2)/2
HG = Wm1 - H
hG = (C-B)/2
Mop = HD*hD + HT*hT + HG*hG
print( f'For the operating conditions, the total flange moment Mop = {Mop} poundxin')

For the operating conditions, the total flange moment Mop = 4285.980240735151 poundxin


For gasket seating, the total flange moment is (Eq. 2-6(6))

$M_a = W\frac{C-G}{2}$

where $M_a$ has the same notation in the Cryofab analysis ($M_o$ in ASME)

In [10]:
Ma = W*(C-G)/2
print( f'For gasket seating, the total flange moment Ma = {Ma} poundxin')

For gasket seating, the total flange moment Ma = 4413.990999810149 poundxin


Appendix 2-6:

> When the bolt spacing exceeds $2a + t$, multiply $M_O$ by the bolt spacing correction factor $B_{SC}$ for calculating flange stress, where
>
> $B_{SC} = \sqrt{\frac{B_S}{2a+t}}$

In [11]:
Bsc = np.sqrt(Bs/(2*a+t))
print( f'The bolt spacing correction factor, Bsc = {Bsc}')

The bolt spacing correction factor, Bsc = 2.275880586593231


### Tangential Stress in Flange

Appendix 2-7(b)(11):

$S_T = \frac{YM_o}{t^2B}$

In our case, we need to apply the bolt spacing correction factor, and $M_o = M_{op}$ (the operating conditions), therefore,

$S_T = \frac{YM_{op}B_{SC}}{t^2B}$

According to 2-3 and Figure 2-7.1,

$Y = \frac{1}{K-1}[0.66845+5.7169\frac{K^2\log_{10}K}{K^2-1}]$

where $K$ = ratio of outside diameter of flange to inside diameter of flange
= $A/B$

In [12]:
K = A/B
Y = 1./(K-1)*(0.66845+5.7169*(K**2)*np.log10(K)/((K**2)-1))
ST = Y*Mop*Bsc/((t**2)*B)
# print( f'K = {K}, Y = {Y}')
print( f'Tangential stress is {ST} psi')

Tangential stress is 13811.140777819619 psi


S: maximum allowable stress value in tension from applicable table of stress values referenced by UG-23, 20 kpi = 20000 psi, from Table ULT-23, at temperature of 0, 100, 150 F

In [13]:
S = 20000
isSTOK = False
if ST < S:
    isSTOK = True
print(f'Maximum allowable stress = {S} psi, the tangential stress OK? {isSTOK}')

Maximum allowable stress = 20000 psi, the tangential stress OK? True
